In [1]:
# standard libs
import os
import glob
import random
import numpy as np
import pandas as pd

# formatting and setup
%matplotlib notebook
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200

# get all the sensitivities
sensitivities_dir = './sensitivities/'
output_dir = '.'

rundate =  '2021-03-05'


In [9]:
from string import digits

CR_correlation = pd.read_csv('FRTB_CR_Correlation_Revised.csv', index_col=0)
CR_buckets = pd.read_csv('FRTB_CR_Cpy.csv', index_col=0)
CM_buckets = pd.read_csv('FRTB_CM_Buckets.csv', index_col=0)
CM_map = pd.read_csv('comm.csv', index_col=0)
CM_lookup = CM_map['Class'].to_dict()

EQ_buckets = pd.read_csv('FRTB_EQ_Buckets.csv', index_col=0)
EQ_map = pd.read_csv('EQ_Buckets.csv')
EQ_lookup = EQ_map.set_index(EQ_map['Equities'].apply(lambda x:x.replace('/','_')))['Desc'].to_dict()
CR_Credit_Spread = pd.read_csv('CP_mapping.csv', index_col=0)

IR_Delta_Correlation = pd.read_csv('FRTB_IR_Correlation.csv', index_col=0)
IR_Delta_Tenors = np.array([1, 2, 5, 10, 30])
CR_Delta_Tenors = np.array([.5, 1, 3, 5, 10])

IR_Delta_Curr = ['ZAR','USD','JPY','EUR','GBP','AUD','CAD','SEK']
IR_Delta_RW = {'1 year':0.0159,
               '2 years':0.0133,
               '5 years':0.0106,
               '10 years':0.0106,
               '30 years':0.0106,
               'Inflation':0.0159}
               
no_digits=str.maketrans('', '', digits)

# stuff from kudsai
CR_Credit_Spread = pd.read_csv('CP_mapping.csv', index_col=0)

CR_bucket_map = {'1a':'Sovereigns_a','1b':'Sovereigns_b','2':'Financials',
                 '3':'Industrials','4':'Consumer Goods','5':'Technology',
                 '6':'Utilities','7':'Other'}

CR_Credit_Spread['Code']=CR_Credit_Spread.apply(lambda x:('IG_' if x['Bucket']=='IG' else 'HYNR_')+CR_bucket_map[x['Reg CVA Bucket']], axis=1)
Kudsai = CR_Credit_Spread['Code'].to_dict()

In [10]:
CR_correlation

,Sovereigns_b,Financials,Industrials,Consumer Goods,Technology,Utilities,Other
Crossbucket correlation,,,,,,,
Sovereigns_b,1.00,0.10,0.20,0.25,0.20,0.15,0
Financials,0.10,1.00,0.05,0.15,0.20,0.05,0
Industrials,0.20,0.05,1.00,0.20,0.25,0.05,0
Consumer Goods,0.25,0.15,0.20,1.00,0.25,0.05,0
Technology,0.20,0.20,0.25,0.25,1.00,0.05,0
Utilities,0.15,0.05,0.05,0.05,0.05,1.00,0
Other,0.00,0.00,0.00,0.00,0.00,0.00,1


In [13]:
# need to interpolate the IR curves at the tenors specified

def calc_bucket_sensi(delta_tenors, val):
    max_tenor = val.index.max()
    deltas = delta_tenors[:delta_tenors.searchsorted(max_tenor)+1]
    x = val.index.values
    y = val.iloc[:,-1].values
    bump = []
    prev_tenor = 0.0
    for index, tenor in enumerate(deltas):
        prev_piece = (x>=prev_tenor)&(x<=tenor)
        next_tenor = delta_tenors[index+1] if index<delta_tenors.size-1 else max_tenor
        next_piece = (x>=tenor)&(x<=next_tenor)

        prev_sensi = (np.interp(x[prev_piece], [prev_tenor,tenor],[0.0, 0.0001])*y[prev_piece]).sum()
        next_sensi = (np.interp(x[next_piece], [tenor,next_tenor],[0.0001, 0.0])*y[next_piece]).sum()

        prev_tenor = tenor
        bump.append((prev_sensi+next_sensi)/0.0001)
    return bump, deltas
    
    
def interpolate_IR_Curves(df, crb_name):
    q = df.set_index(['Rate','Tenor'])
    md = []
    for rate in q.index.levels[0]:
        val = q.loc[rate]
        cpy = val.columns[-1]
        name = (rate.split('/')[1] if '/' in rate else rate).split('.')
        if name[0].startswith('InterestRate') or name[0].startswith('InflationRate'):
            
            if name[1][:3] in IR_Delta_Curr:
                bump, ir_deltas = calc_bucket_sensi(IR_Delta_Tenors, val)
                r=pd.DataFrame({'Rate':rate, 'Tenor2':0, 'Tenor3':0, cpy:bump}, index=ir_deltas )
                r.index.name='Tenor'
            else:
                # just a straight sum
                r=pd.DataFrame({'Rate':rate, 'Tenor2':0, 'Tenor3':0, cpy:val.iloc[:,-1].sum()}, index=[0.0] )
                r.index.name='Tenor'
                
        elif name[0].startswith('SurvivalProb'):
            # assume recovery is 0.5
            # here we get sensitivities to -log(survival)
            # need to change it to CDS spreads via approximate formula
            # S = ((1-R)/t)*-log(survivalProb)
            
            bump, cr_deltas = calc_bucket_sensi(CR_Delta_Tenors, val)
            # new_rate = 'Static_Input/SurvivalProb.{}'.format(crb_name)
            new_rate = 'SurvivalProb.{}'.format(crb_name)
            r=pd.DataFrame({'Rate':new_rate, 'Tenor2':0, 'Tenor3':0, cpy:np.array(bump)*np.array(0.5/cr_deltas)}, index=cr_deltas )
            r.index.name='Tenor'
        else:
            r = val
            r['Rate']=rate

        md.append(r)

    return pd.concat(md, sort=True).reset_index().set_index(['Rate','Tenor','Tenor2','Tenor3']) if md else df

cva_default = pd.read_csv(sensitivities_dir+'CVADEFAULT_Stats_'+rundate+'_Total.csv', index_col=0)


In [27]:
delta={}
for crb in glob.glob(sensitivities_dir+'CVA_'+rundate+'*.csv'):
    filename = os.path.split(crb)[1]    
    delta[filename[15:-4]]= interpolate_IR_Curves(pd.read_csv(crb).iloc[:,:-1], filename[15:-4])

In [76]:
np.interp( [1.26 , 1.51 , 1.75 , 2.00], [1,2], [0.0001, 0.0] )

array([7.4e-05, 4.9e-05, 2.5e-05, 0.0e+00])

In [73]:
delta['CrB_Kathu_Solar_Park_ISDA']

CrB_Kathu_Solar_Park_ISDA
Rate                                               Tenor Tenor2 Tenor3                           
GBMAssetPriceTSModelParameters.ZAR.Vol             0.01  0.00   0.00                    -1,600.29
                                                   0.02  0.00   0.00                    -4,289.92
                                                   0.04  0.00   0.00                   -21,021.12
                                                   0.09  0.00   0.00                   -71,938.41
                                                   0.17  0.00   0.00                  -141,262.30
                                                   0.26  0.00   0.00                  -251,779.02
                                                   0.51  0.00   0.00                   -70,663.44
                                                   0.76  0.00   0.00                  -176,819.34
                                                   1.01  0.00   0.00                  -299,112.20
                                                   1.26  0.00   0.00                  -222,826.52
                                                   1.51  0.00   0.00                  -153,276.55
                                                   2.01  0.00   0.00                   -80,990.29
                                                   3.01  0.00   0.00                  -171,693.12
                                                   4.01  0.00   0.00                  -176,233.84
                                                   5.00  0.00   0.00                   -89,289.52
                                                   7.01  0.00   0.00                   -38,420.59
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Al... 0.00  0.00   0.00                   -12,492.29
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Al... 0.00  0.00   0.00                -4,385,524.40
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Co... 0.00  0.00   0.00                  -157,385.67
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Qu... 0.00  0.00   0.00                    96,703.26
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Qu... 0.00  0.00   0.00                   936,483.30
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Si... 0.00  0.00   0.00                   102,317.61
                                                   0.08  0.00   0.00                   269,762.97
                                                   0.25  0.00   0.00                   446,910.85
                                                   0.50  0.00   0.00                   348,634.38
                                                   1.00  0.00   0.00                  -810,021.10
                                                   2.00  0.00   0.00                -1,488,600.10
                                                   4.00  0.00   0.00                 3,317,962.40
                                                   6.00  0.00   0.00                -4,407,022.40
                                                   8.00  0.00   0.00                  -138,944.64
                                                   10.00 0.00   0.00                   -15,416.21
HullWhite2FactorModelParameters.ZAR-JIBAR-3M.Si... 0.00  0.00   0.00                  -402,390.55
                                                   0.08  0.00   0.00                 1,048,853.90
                                                   0.25  0.00   0.00                 2,590,856.60
                                                   0.50  0.00   0.00                   454,010.60
                                                   1.00  0.00   0.00                14,657,892.80
                                                   2.00  0.00   0.00                 7,394,184.80
                                                   4.00  0.00   0.00                12,372,132.80
                                                   6.00  0.00   0.00                 2,791,735.80
                                                   8.0

In [28]:
all_cva=cva_default.fillna(0.0).to_dict()['CVA']

## Load up parameters

- Load up Risk weights by risk type (FX, Commodity, Counterparty, Equity, Interest Rate)
- Load up Correlations for some risk types

## IR Vega

For IR Vega per currency, the change in the calibration parameters is loaded up and the CVA change is calculated (and then divided by 1%)

In [29]:
IR_Vega=pd.read_csv('IRVega.csv', index_col=0)
IR_Vega_Lookup = IR_Vega.T.to_dict()

## Tenor Buckets

- IR data needs to be bucketed into the following (for Domestic, USD,JPY, EUR, GBP, AUD, CAD, SEK) :
  - 1 year
  - 2 years
  - 5 years
  - 10 years
  - 30 years
  
- Counterparty spread data needs the following buckets:
  - 6 months
  - 1 year
  - 3 years
  - 5 years
  - 10 years
  
FX, Commodities and Equites do not have tenors  

In [30]:
def apply_vega(row):    
    if row.name[0].startswith('HullWhite2FactorModelParameters') and len(row)==1:
        clipped = row[0]
        if row.name[0].endswith('.Alpha_2'):
            clipped = row[0].clip(-10000000,10000000)
        rate =  row.name[0].split('.')
        return 100.0*IR_Vega_Lookup[rate[1]].get(rate[2],0.0)*clipped
    else:
        return row[0]
    
def bucket_ir(y):
    if y==1:
        return '1 year'
    elif y==2:
        return '2 years'
    elif y==5:
        return '5 years'
    elif y==10:
        return '10 years'
    elif y==30:
        return '30 years'
    else:
        return 'all'
    
def bucket_cr(y):
    if y==.5:
        return '6 months'
    elif y==1:
        return '1 year'
    elif y==3:
        return '3 year'
    elif y==5:
        return '5 year'
    else:
        return '10 year'
    
def classify(x):
    if x['Rate'].startswith('HullWhite2FactorModelParameters'):
        name = x['Rate'].split('.')
        return 'Vega.IR.'+name[1][:3]
    else:
        name = (x['Rate'].split('/')[1] if '/' in x['Rate'] else x['Rate']).split('.')
        if name[0]=='InterestRate':
            curr = name[1][:3] 
            return 'Delta.IR.'+curr+'.'+bucket_ir(x['Tenor'])
        elif name[0]=='InflationRate':
            curr = name[1][:3] 
            return 'Delta.IR.'+curr+'.Inflation'
        elif name[0]=='FxRate':
            curr = name[1][:3] 
            return 'Delta.FX.'+curr+'.USD'
        elif name[0]=='SurvivalProb':
            sector = Kudsai.get(name[1],'HYNR_Other')
            return 'Delta.CR.'+sector+'.'+bucket_cr(x['Tenor'])
        elif name[0]=='FXVol':
            curr = name[1:]
            return 'Vega.FX.'+'.'.join(curr)
        elif name[0]=='InterestRateVol':
            curr = name[1][:3] 
            return 'Vega.IR.'+curr
        elif name[0]=='ForwardPrice':
            com = name[1]
            return 'Delta.CM.'+CM_lookup[com]
        elif name[0]=='ForwardPriceVol':
            com = name[1]
            return 'Vega.CM.'+CM_lookup[com]
        elif name[0]=='EquityPrice':
            eq = name[1].translate(no_digits)
            return 'Delta.EQ.'+EQ_lookup.get(eq,'Other sector')
        elif name[0]=='EquityPriceVol':
            eq = name[1].translate(no_digits)
            return 'Vega.EQ.'+EQ_lookup.get(eq,'Other sector')

FRTB_buckets = {}        
for k,z in delta.items():
    if z.values.shape[0]:
        # strip out the fx implied and make sure the sigmas are close to zero
        v= pd.concat( [ z[(z.index.get_level_values(0).str.startswith('HullWhite') & (z.index.get_level_values(1)==0.0))], 
                        z[~z.index.get_level_values(0).str.startswith('GBMAssetPriceTSModelParameters')] ] )
        v.iloc[:,0] = v.apply(apply_vega, axis=1)
        v['FRTB_Buckets'] = v.index.to_frame().apply(classify, axis=1)
        FRTB_buckets[k]=v.groupby('FRTB_Buckets').agg(np.sum)

In [31]:
all_buckets = pd.concat(FRTB_buckets.values(), axis=1, sort=True)

In [33]:
for i in sorted(all_buckets.fillna(0.).columns):
    print(i)


CrB_1st_Contact_Money__Pty__Limited_NonISDA
CrB_3G_Mobile_NonISDA
CrB_ABAX_Investments_ISDA
CrB_ABAX_Nedbank_Bal_Presc_Fd_NonISDA
CrB_ABAX_SBSA_Nedgroup_Inv_Flex_Inc_F_NonISDA
CrB_ABSA_Bank_Jhb_ISDA
CrB_ACWA_Power_Solafri_Bkp_CSP_PP_RF_NonISDA
CrB_AECI_Limited_ISDA
CrB_AVI_Financial_Services__Pty__Limited_ISDA
CrB_Abax_Global_Income_Fund_NonISDA
CrB_Abax_Ned_Abax_Fxd_Int_Pres_RI_Hd_Fd_NonISDA
CrB_Abax_Nedbank_itf_Div_Inc_Presc_Fund_NonISDA
CrB_Accelerate_Property_Fund_ISDA
CrB_Accelerate_Property_Fund_NonISDA
CrB_Afgri_Operations__Pty__Limited_ISDA
CrB_Africa_Health_Research_Ins_ISDA
CrB_Africa_X-Ray_Industrial_and_Medical_Ltd_NonISDA
CrB_African_Resonance_Business_Solution_NonISDA
CrB_Afrox_ISDA
CrB_Altron_Finance__Pty__Ltd_ISDA
CrB_Anchor_Private_Clients_NonISDA
CrB_Anglo_SA_Finance_NonISDA
CrB_ArcelorMittal_SA_ISDA
CrB_ArrowGem_Ltd_ISDA
CrB_Aurora_Wind_Power__RF___Pty__Ltd_ISDA
CrB_Aurum_Institute_ISDA
CrB_AutoX_ISDA
CrB_Avon_Peaking_Power_ISDA
CrB_Axiz__Pty__Ltd_NonISDA
CrB_BIC_Sou

In [78]:
#all_buckets
all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.FX')]]

,CrB_1st_Contact_Money__Pty__Limited_NonISDA,CrB_3G_Mobile_NonISDA,CrB_Abax_Global_Income_Fund_NonISDA,CrB_ABAX_Investments_ISDA,CrB_ABAX_Nedbank_Bal_Presc_Fd_NonISDA,CrB_Abax_Nedbank_itf_Div_Inc_Presc_Fund_NonISDA,CrB_Abax_Ned_Abax_Fxd_Int_Pres_RI_Hd_Fd_NonISDA,CrB_ABAX_SBSA_Nedgroup_Inv_Flex_Inc_F_NonISDA,CrB_ABSA_Bank_Jhb_ISDA,CrB_Accelerate_Property_Fund_ISDA,...,CrB_Vukile_Property_Fun_ISDA,CrB_Vukile_Property_Fun_NonISDA,CrB_Warshay_Investments__Pty__Limited_ISDA,CrB_Wealthpoint_Capital_NonISDA,CrB_Western_Platinum_Limited_ISDA,CrB_Windfall_59_Properties_RF_NonISDA,CrB_Woolworths_ISDA,CrB_WWC_Asset_Management-GTC_Flex_Fund_ISDA,CrB_Zimbel_Business_Enterprises__Pty__L_ISDA,CrB_Zonke_Engineering__Pty__Limited_NonISDA
Delta.FX.AED.USD,"37,131.10",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.AUD.USD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.BWP.USD,nan,"67,361.37",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.CAD.USD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.CHF.USD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.CNH.USD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.DKK.USD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.EUR.USD,"-1,594.91","-2,385.87","52,911.39","28,217.18",nan,"5,964.30",nan,"62,405.69",nan,"5,307,127.20",...,"7,107,846.40","3,327,124.80","2,287,339.00",nan,nan,nan,nan,nan,"13,027.16","-23,388.50"
Delta.FX.GBP.USD,"-3,693.41",nan,"15,000.42","25,683.17",nan,nan,-752.77,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Delta.FX.HKD.USD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Vega FX

- FX has no term structure with a risk weight of $.55\sqrt{4}$

## Vega IR

- IR sensitivity is obtained by sensitivity to the Calibration parameters (not the vol surface) - so we need to scale this down
 - has a risk weight of $.55\sqrt{6}$
 
## Vega CM

 - CM  has a risk weight of $.55\sqrt{12}$
 
## Vega EQ

 - EQ has a risk weight of $.55\sqrt{2}$ for large cap buckets and $.55\sqrt{6}$ for small cap buckets

In [35]:
vega_fx = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.FX')]].sum(axis=1)
vega_ir = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.IR')]].sum(axis=1)
vega_cm = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.CM')]].sum(axis=1)
vega_eq = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.EQ')]].sum(axis=1)

# constants as laid out above
R = 0.01 # Hedge disallowance
vega_IR_RW = .55*np.sqrt(6)
vega_FX_RW = .55*np.sqrt(4)
vega_CM_RW = .55*np.sqrt(12)
vega_EQ_L_RW = .55*np.sqrt(2)
vega_EQ_S_RW = .55*np.sqrt(6)

In [36]:
all_buckets.loc['Vega.IR.USD'].abs().sort_values()

CrB_Nedbank_itf_Prescient_Core_Equi_Fnd_ISDA                  0.00
CrB_RGBrose_Automotive_Components_Ltd_NonISDA                 0.00
CrB_Service_First_cc_NonISDA                                  0.00
CrB_Graaffs_Trust_Limited_NonISDA                             0.00
CrB_Valufin__Pty__Limited_NonISDA                             0.00
CrB_SFX_Dealing_Room_NonISDA                                  0.00
CrB_Karl_Storz_Endoscopy_SA_NonISDA                           0.00
CrB_BMS_Foods_NonISDA                                         0.00
CrB_Afrox_ISDA                                                0.00
CrB_Open_Box_Software__Pty__Limited_NonISDA                   0.00
CrB_Meso_Capital_NonISDA                                      0.00
CrB_Larson_Industries_CC_NonISDA                              0.00
CrB_Pieter_Nicolaas_De_Waal_ISDA                              0.00
CrB_Finest_Monetary_Solutions_cc_NonISDA                      0.00
CrB_Global_Intermediate_Traders_NonISDA                       

In [37]:
bucket_vega_ir = pd.DataFrame({'SA-WS (CVA)':vega_IR_RW*vega_ir, 'SA-WS (Hedge)':vega_IR_RW*0.0}).set_index(vega_ir.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_ir['SA-WS']=bucket_vega_ir['SA-WS (CVA)']+bucket_vega_ir['SA-WS (Hedge)']
bucket_vega_ir['K_b']=np.sqrt(bucket_vega_ir['SA-WS']**2 + R*(bucket_vega_ir['SA-WS (Hedge)']**2))
bucket_vega_ir


,SA-WS (CVA),SA-WS (Hedge),SA-WS,K_b
AUD,"1,363,886.98",0.00,"1,363,886.98","1,363,886.98"
CAD,2.43,0.00,2.43,2.43
CHF,"1,002.78",0.00,"1,002.78","1,002.78"
EUR,"11,257,130.35",0.00,"11,257,130.35","11,257,130.35"
GBP,"-393,532.36",0.00,"-393,532.36","393,532.36"
JPY,-77.63,0.00,-77.63,77.63
USD,"26,767,785.98",0.00,"26,767,785.98","26,767,785.98"
ZAR,"-1,543,654.91",0.00,"-1,543,654.91","1,543,654.91"


In [38]:
bucket_vega_fx = pd.DataFrame({'SA-WS (CVA)':vega_FX_RW*vega_fx, 'SA-WS (Hedge)':vega_FX_RW*0.0}).set_index(vega_fx.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_fx['SA-WS']=bucket_vega_fx['SA-WS (CVA)']+bucket_vega_fx['SA-WS (Hedge)']
bucket_vega_fx['K_b']=np.sqrt(bucket_vega_fx['SA-WS']**2 + R*(bucket_vega_fx['SA-WS (Hedge)']**2))
bucket_vega_fx


,SA-WS (CVA),SA-WS (Hedge),SA-WS,K_b
EUR.ZAR,"34,701.61",0.00,"34,701.61","34,701.61"
GBP.ZAR,"17,002.11",0.00,"17,002.11","17,002.11"
USD.ZAR,"6,699,485.26",0.00,"6,699,485.26","6,699,485.26"


In [58]:
bucket_vega_cm = pd.DataFrame({'SA-WS (CVA)':vega_CM_RW*vega_cm, 'SA-WS (Hedge)':vega_CM_RW*0.0}).set_index(vega_cm.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_cm['SA-WS']=bucket_vega_cm['SA-WS (CVA)']+bucket_vega_cm['SA-WS (Hedge)']
bucket_vega_cm['K_b']=np.sqrt(bucket_vega_cm['SA-WS']**2 + R*(bucket_vega_cm['SA-WS (Hedge)']**2))
bucket_vega_cm


,SA-WS (CVA),SA-WS (Hedge),SA-WS,K_b


In [40]:
vega_EQ_RW = vega_eq.index.map(lambda x:vega_EQ_L_RW if x.split('.')[2].startswith('L') else vega_EQ_S_RW)
bucket_vega_eq = pd.DataFrame({'SA-WS (CVA)':vega_EQ_RW*vega_eq, 
                               'Risk Weight':vega_EQ_RW,
                               'SA-WS (Hedge)':vega_EQ_RW*0.0}).set_index(vega_eq.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_eq['SA-WS']=bucket_vega_eq['SA-WS (CVA)']+bucket_vega_eq['SA-WS (Hedge)']
bucket_vega_eq['K_b']=np.sqrt(bucket_vega_eq['SA-WS']**2 + R*(bucket_vega_eq['SA-WS (Hedge)']**2))
bucket_vega_eq


,SA-WS (CVA),Risk Weight,SA-WS (Hedge),SA-WS,K_b
L EM Consumer,-1.05,0.78,0.00,-1.05,1.05
L EM Financials,"68,696.89",0.78,0.00,"68,696.89","68,696.89"
Other sector,"34,171.42",1.35,0.00,"34,171.42","34,171.42"
S EM All,"-3,162.68",1.35,0.00,"-3,162.68","3,162.68"


##  Delta

Now calculate all the delta contributions for each risk factor type


###  Commodity Delta

- All CM Need to be bucketed as per the regs

In [41]:
CM_RW = CM_buckets.set_index('Commodity Group').to_dict()['Risk Weight']
delta_cm = pd.DataFrame({'Delta_CM':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.CM')]].sum(axis=1)})
delta_cm['RiskWeight']=delta_cm.apply(lambda x:CM_RW[x.name.split('.')[2]], axis=1)
delta_cm['SA-WS (Hedge)']=0.0
delta_cm['SA-WS (CVA)']=delta_cm['RiskWeight']*delta_cm['Delta_CM']
delta_cm['SA-WS']=delta_cm['SA-WS (CVA)']+delta_cm['SA-WS (Hedge)']

In [42]:
bucket_delta_cm = delta_cm.set_index(delta_cm.index.map(lambda x:x.split('.',2)[2]))
bucket_delta_cm['K_b']=np.sqrt((1-0.01)*bucket_delta_cm['SA-WS']**2 +
                               0.01*(bucket_delta_cm['SA-WS (Hedge)']**2+bucket_delta_cm['SA-WS (CVA)']**2))
bucket_delta_cm

,Delta_CM,RiskWeight,SA-WS (Hedge),SA-WS (CVA),SA-WS,K_b
Crude Oil,-4.16,0.35,0.00,-1.46,-1.46,1.46
Metals,-13.48,0.40,0.00,-5.39,-5.39,5.39
Precious Metals,727.73,0.20,0.00,145.55,145.55,145.55


###  Equity Delta

- All EQ Need to be bucketed as per the regs

In [43]:
EQ_RW = EQ_buckets.set_index('Equity Group').to_dict()['Risk Weight']
delta_eq = pd.DataFrame({'Delta_EQ':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.EQ')]].sum(axis=1)})
delta_eq['RiskWeight'] = delta_eq.apply(lambda x:EQ_RW[x.name.split('.')[2]], axis=1)
delta_eq['SA-WS (Hedge)']=0.0
delta_eq['SA-WS (CVA)']=delta_eq['RiskWeight']*delta_eq['Delta_EQ']
delta_eq['SA-WS']=delta_eq['SA-WS (CVA)']+delta_eq['SA-WS (Hedge)']

In [44]:
bucket_delta_eq = delta_eq.set_index(delta_eq.index.map(lambda x:x.split('.',2)[2]))
bucket_delta_eq['K_b']=np.sqrt((1-0.01)*bucket_delta_eq['SA-WS']**2 +
                               0.01*(bucket_delta_eq['SA-WS (Hedge)']**2+bucket_delta_eq['SA-WS (CVA)']**2))
bucket_delta_eq

,Delta_EQ,RiskWeight,SA-WS (Hedge),SA-WS (CVA),SA-WS,K_b
L AE Consumer,-0.40,0.30,0.00,-0.12,-0.12,0.12
L AE Financials,122.64,0.50,0.00,61.32,61.32,61.32
L AE Materials,68.51,0.40,0.00,27.41,27.41,27.41
L AE Telecoms,-93.45,0.35,0.00,-32.71,-32.71,32.71
L EM Consumer,-508.73,0.55,0.00,-279.80,-279.80,279.80
L EM Financials,"4,918.30",0.55,0.00,"2,705.07","2,705.07","2,705.07"
L EM Materials,46.62,0.45,0.00,20.98,20.98,20.98
L EM Telecoms,-16.47,0.60,0.00,-9.88,-9.88,9.88
Other sector,"3,728.42",0.70,0.00,"2,609.89","2,609.89","2,609.89"
S EM All,"-18,112.51",0.70,0.00,"-12,678.76","-12,678.76","12,678.76"


###  FX Delta

- All fx delta's have the same risk weight (.21)
- bucketed by currency pair

In [45]:
delta_fx = pd.DataFrame({'Delta_FX':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.FX')]].sum(axis=1)})
delta_fx['RiskWeight']=.21
delta_fx['SA-WS (Hedge)']=0.0
delta_fx['SA-WS (CVA)']=delta_fx['RiskWeight']*delta_fx['Delta_FX']
delta_fx['SA-WS']=delta_fx['SA-WS (CVA)']+delta_fx['SA-WS (Hedge)']

In [46]:
bucket_delta_fx = delta_fx.set_index(delta_fx.index.map(lambda x:x.split('.',2)[2]))
bucket_delta_fx['K_b']=np.sqrt(bucket_delta_fx['SA-WS']**2 + R*(bucket_delta_fx['SA-WS (Hedge)']**2))
bucket_delta_fx

,Delta_FX,RiskWeight,SA-WS (Hedge),SA-WS (CVA),SA-WS,K_b
AED.USD,"36,814.99",0.21,0.00,"7,731.15","7,731.15","7,731.15"
AUD.USD,"24,843,709.06",0.21,0.00,"5,217,178.90","5,217,178.90","5,217,178.90"
BWP.USD,"60,128.05",0.21,0.00,"12,626.89","12,626.89","12,626.89"
CAD.USD,857.40,0.21,0.00,180.05,180.05,180.05
CHF.USD,"-21,182.61",0.21,0.00,"-4,448.35","-4,448.35","4,448.35"
CNH.USD,"-14,904,581.38",0.21,0.00,"-3,129,962.09","-3,129,962.09","3,129,962.09"
DKK.USD,69.37,0.21,0.00,14.57,14.57,14.57
EUR.USD,"40,261,399.90",0.21,0.00,"8,454,893.98","8,454,893.98","8,454,893.98"
GBP.USD,"1,744,813.92",0.21,0.00,"366,410.92","366,410.92","366,410.92"
HKD.USD,"-227,005.62",0.21,0.00,"-47,671.18","-47,671.18","47,671.18"


## Counterparty Delta

- need to group counterparties into the following buckets:
  - HYNR_Consumer Goods
  - HYNR_Financials	
  - HYNR_Industrials
  - HYNR_Sovereigns	(a and b)
  - HYNR_Technology	
  - HYNR_Utilities	
  - HYNR Other
  - IG_Consumer Goods
  - IG_Financials	
  - IG_Industrials	
  - IG_Sovereigns (a and b)
  - IG_Technology	
  - IG_Utilities	
  - IG Other

- Then need to apply correlation within tenors of the same counterparty (.9) vs  
   - correlation of tenors with other counterparties of same credit quality (.5 for same tenors, .45 for different tenors)
   - correlation of tenors with other counterparties of different credit quality (.4 for same tenors, .36 for different tenors)
- currently ignoring legally related entities

- Then apply the Hedge disallowance parameter (0.01)

In [49]:
delta_cr = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.CR')]]

def CR_RW(row):
    bucket = row.name.split('.')[2]
    return CR_buckets.loc[bucket]
    
def CR_reindex(df):
    return df.set_index(
        pd.MultiIndex.from_arrays(
            [df.index.map(lambda x:x.split('.')[2]),df.index.map(lambda x:'.'.join(x.split('.')[3:]))]))
    
CR_SA_WS_CVA = delta_cr.apply(CR_RW, axis=1).values * delta_cr
CR_SA_WS_hedge = 0.0 * delta_cr
CR_SA_WS = CR_SA_WS_CVA+CR_SA_WS_hedge

delta_cr_index=CR_reindex(CR_SA_WS)
delta_cr_hedge_index=CR_reindex(CR_SA_WS_hedge)
delta_cr_CVA_index=CR_reindex(CR_SA_WS_CVA)

all_delta_cr = {}

for CP_group, CP_Classifications in cp_buckets_map.items():
    group0 = delta_cr_index.xs(CP_Classifications[0]).dropna(how='all', axis=1)
    
    if len(CP_Classifications)>1:
        group1=delta_cr_index.xs(CP_Classifications[1]).dropna(how='all', axis=1)
        tenor_index = np.union1d(group0.index, group1.index)
        delta_cp_size = np.array([group0.reindex(tenor_index).size,group1.reindex(tenor_index).size])//tenor_index.size
    else:        
        tenor_index = group0.index
        delta_cp_size = np.array([group0.size])//tenor_index.size
        
    corr_cp      = []
    del_cp       = []
    del_cp_cva   = []
    del_cp_hedge = []
    correlation  = []
    
    for index, CP_Classification in enumerate(CP_Classifications):
        tenor_size = tenor_index.size
        delta_cp = delta_cr_index.xs(CP_Classification).dropna(
            how='all', axis=1).reindex(tenor_index).fillna(0.0).values.reshape(-1,1)
        delta_cp_hedge = delta_cr_hedge_index.xs(CP_Classification).dropna(
            how='all', axis=1).reindex(tenor_index).fillna(0.0).values.reshape(-1,1)
        delta_cp_cva = delta_cr_CVA_index.xs(CP_Classification).dropna(
            how='all', axis=1).reindex(tenor_index).fillna(0.0).values.reshape(-1,1)
        
        del_cp.append(delta_cp)
        del_cp_hedge.append(delta_cp_hedge)
        del_cp_cva.append(delta_cp_cva)

    # matrix for diagonal
    diag = np.ones((tenor_size,tenor_size))*.9
    np.fill_diagonal(diag,1)

    # matrix for off diagonal - same Group
    off_diag_same = np.ones((tenor_size,tenor_size))*.45
    np.fill_diagonal(off_diag_same, .5)

    # matrix for off diagonal - different Group
    off_diag_other = np.ones((tenor_size,tenor_size))*.36
    np.fill_diagonal(off_diag_other, .4)

    for i in range(sum(delta_cp_size)):
        rows = [diag if j==i else (off_diag_same if i<delta_cp_size[0] else off_diag_other) for j in range(sum(delta_cp_size))]
        correlation.append(np.concatenate(rows, axis=1))
        
    corr = np.concatenate(correlation, axis=0)
    delta_cp_full = np.concatenate(del_cp,axis=0)
    delta_cp_cva_full = np.concatenate(del_cp_cva,axis=0)
    delta_cp_hedge_full = np.concatenate(del_cp_hedge,axis=0)
    
    weight = delta_cp_full.T.dot(corr).dot(delta_cp_full)

    all_delta_cr[CP_group]={'WS_k(CVA)^2' : (delta_cp_cva_full*delta_cp_cva_full).sum(),
                                     'WS_k(Hdg)^2' : (delta_cp_hedge_full*delta_cp_hedge_full).sum(),
                                     'WS_k^2' : weight[0][0]}
                                     
        
# make sure the index matches the correlation matrix                                    
bucket_delta_cr = pd.DataFrame(all_delta_cr).T.reindex(CR_correlation.index)

In [48]:
# get the bucket maps
cp_buckets_map = {}
for i in delta_cr_index.index.levels[0]:
    cp_buckets_map.setdefault(i.split('_',1)[1],[]).append(i)


In [50]:
bucket_delta_cr

,WS_k(CVA)^2,WS_k(Hdg)^2,WS_k^2
Crossbucket correlation,,,
Sovereigns_b,"18,867,248,035.44",0.00,"29,565,716,735.19"
Financials,"381,353,304,309,173.56",0.00,"869,925,039,124,189.88"
Industrials,"89,651,178,660.46",0.00,"587,793,052,311.61"
Consumer Goods,"7,382,532,022,536.17",0.00,"26,445,222,477,990.45"
Technology,"99,850,773,846.99",0.00,"212,849,060,822.21"
Utilities,"336,320,164,894.25",0.00,"2,086,545,500,748.46"
Other,"17,971,882,501,914.73",0.00,"62,795,397,048,190.28"


In [51]:
bucket_delta_cr['Hedge disallowance(R)']=0.01
bucket_delta_cr['K_b']=np.sqrt(
    bucket_delta_cr['WS_k^2']*( 
        1-bucket_delta_cr['Hedge disallowance(R)'])+bucket_delta_cr['Hedge disallowance(R)']*(
        bucket_delta_cr['WS_k(CVA)^2']+bucket_delta_cr['WS_k(Hdg)^2'])
)

In [52]:
bucket_delta_cr

,WS_k(CVA)^2,WS_k(Hdg)^2,WS_k^2,Hedge disallowance(R),K_b
Crossbucket correlation,,,,,
Sovereigns_b,"18,867,248,035.44",0.00,"29,565,716,735.19",0.01,"171,635.46"
Financials,"381,353,304,309,173.56",0.00,"869,925,039,124,189.88",0.01,"29,411,550.82"
Industrials,"89,651,178,660.46",0.00,"587,793,052,311.61",0.01,"763,421.01"
Consumer Goods,"7,382,532,022,536.17",0.00,"26,445,222,477,990.45",0.01,"5,123,923.85"
Technology,"99,850,773,846.99",0.00,"212,849,060,822.21",0.01,"460,129.41"
Utilities,"336,320,164,894.25",0.00,"2,086,545,500,748.46",0.01,"1,438,416.92"
Other,"17,971,882,501,914.73",0.00,"62,795,397,048,190.28",0.01,"7,896,021.90"


## Interest Rate Delta

- need to group Interest Rates by Currency
- need to then apply correct Risk weight for ZAR (Domestic), USD, JPY, EUR, GBP, AUD and CAD curves
- once we have the risk weights, need to allow for correlation between different tenors
- Then apply the Hedge disallowance parameter (0.01)

In [53]:
delta_ir = pd.DataFrame({'Delta_IR':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.IR')]].sum(axis=1)})

#IR_Delta_RW
def RW(row):
    curr, bucket = row.name.split('.')[-2:]
    if curr in IR_Delta_Curr:
        return IR_Delta_RW[bucket]
    else:
        return 0.0225
    
delta_ir['RiskWeight'] = delta_ir.apply(RW, axis=1)
delta_ir['SA-WS(CVA)'] = delta_ir['RiskWeight'] * delta_ir['Delta_IR']
delta_ir['SA-WS(Hedge)'] = delta_ir['RiskWeight'] * 0.0
delta_ir['SA-WS'] = delta_ir['SA-WS(CVA)']+delta_ir['SA-WS(Hedge)']

delta_ir_index=delta_ir.set_index(pd.MultiIndex.from_arrays([delta_ir.index.map(lambda x:x.split('.')[2]),delta_ir.index.map(lambda x:x.split('.')[3])]))

In [54]:
delta_ir_index

Delta_IR  RiskWeight     SA-WS(CVA)  SA-WS(Hedge)  \
AED all               -1,337.84        0.02         -30.10          0.00   
AUD 1 year       -14,343,352.53        0.02    -228,059.31          0.00   
    10 years      -3,718,160.05        0.01     -39,412.50          0.00   
    2 years      -72,447,453.69        0.01    -963,551.13          0.00   
    30 years        -901,287.55        0.01      -9,553.65          0.00   
    5 years      -49,918,033.20        0.01    -529,131.15          0.00   
BWP all                 -321.84        0.02          -7.24          0.00   
CAD 1 year               -61.93        0.02          -0.98          0.00   
    2 years                0.10        0.01           0.00          0.00   
CHF all               52,632.10        0.02       1,184.22          0.00   
CNH all              661,897.09        0.02      14,892.68          0.00   
DKK all                   -0.13        0.02          -0.00          0.00   
EUR 1 year       -18,179,721.68        0.02    -289,057.57          0.00   
    10 years     -20,929,661.32        0.01    -221,854.41          0.00   
    2 years     -127,571,288.64        0.01  -1,696,698.14          0.00   
    30 years          39,929.33        0.01         423.25          0.00   
    5 years     -118,471,508.43        0.01  -1,255,797.99          0.00   
GBP 1 year        -2,442,148.89        0.02     -38,830.17          0.00   
    10 years            -445.35        0.01          -4.72          0.00   
    2 years       -6,291,585.08        0.01     -83,678.08          0.00   
    5 years         -548,679.26        0.01      -5,816.00          0.00   
HKD all                  776.68        0.02          17.48          0.00   
ILS all                   -1.08        0.02          -0.02          0.00   
JPY 1 year               828.28        0.02          13.17          0.00   
    10 years              -5.88        0.01          -0.06          0.00   
    2 years           -1,604.29        0.01         -21.34          0.00   
    5 years           33,233.21        0.01         352.27          0.00   
MUR all                   -0.06        0.02          -0.00          0.00   
MXN all                    3.76        0.02           0.08          0.00   
NZD all                   43.93        0.02           0.99          0.00   
SEK 1 year                96.02        0.02           1.53          0.00   
    2 years                0.00        0.01           0.00          0.00   
USD 1 year         4,253,162.38        0.02      67,625.28          0.00   
    10 years     -15,068,091.86        0.01    -159,721.77          0.00   
    2 years      130,136,513.45        0.01   1,730,815.63          0.00   
    30 years      -1,382,214.02        0.01     -14,651.47          0.00   
    5 years      232,540,237.66        0.01   2,464,926.52          0.00   
ZAR 1 year       150,515,921.15        0.02   2,393,203.15          0.00   
    10 years    -484,846,527.31        0.01  -5,139,373.19          0.00   
    2 years     -895,966,788.01        0.01 -11,916,358.28          0.00   
    30 years     -12,841,823.66        0.01    -136,123.33          0.00   
    5 years   -1,350,894,414.83        0.01 -14,319,480.80          0.00   
    Inflation       -747,936.89        0.02     -11,892.20          0.00   
ZMW all                   -2.43        0.02          -0.05          0.00   

                       SA-WS  
AED all               -30.10  
AUD 1 year       -228,059.31  
    10 years      -39,412.50  
    2 years      -963,551.13  
    30 years       -9,553.65  
    5 years      -529,131.15  
BWP all                -7.24  
CAD 1 year             -0.98  
    2 years             0.00  
CHF all             1,184.22  
CNH all            14,892.68  
DKK all                -0.00  
EUR 1 year       -289,057.57  
    10 years     -221,854.41  
    2 years    -1,696,698.14  
    30 years          423.25  
    5 years    -1,255,797.99  
GBP 1 year        -38,830.17  
    10 years 

In [55]:
delta_ir_ws = {}
n=IR_Delta_Correlation.index.size
for k,v in delta_ir_index.groupby(level=0):
    delta_ir_ws[k]={'WS_k(CVA)^2':(v['SA-WS(CVA)']**2).sum(),
                    'WS_k(Hdg)^2':(v['SA-WS(Hedge)']**2).sum()}
    
    if k in IR_Delta_Curr:
        block = v.reindex(pd.MultiIndex.from_arrays([[k]*n,IR_Delta_Correlation.index])).fillna(0.0)
        WS = block['SA-WS'].values.reshape(-1,1).dot(block['SA-WS'].values.reshape(1,-1))*IR_Delta_Correlation.values
        delta_ir_ws[k]['WS_k^2']= WS.sum()
    else:
        delta_ir_ws[k]['WS_k^2']= (v['SA-WS']**2).sum()
        
bucket_delta_IR = pd.DataFrame(delta_ir_ws).T


In [56]:
bucket_delta_IR['K_b']=np.sqrt(bucket_delta_IR['WS_k^2']+R*bucket_delta_IR['WS_k(Hdg)^2'])

bucket_delta_IR

,WS_k(CVA)^2,WS_k(Hdg)^2,WS_k^2,K_b
AED,906.09,0.00,906.09,30.10
AUD,"1,262,066,227,615.52",0.00,"2,842,568,437,061.72","1,685,991.83"
BWP,52.44,0.00,52.44,7.24
CAD,0.97,0.00,0.97,0.98
CHF,"1,402,382.36",0.00,"1,402,382.36","1,184.22"
CNH,"221,792,054.23",0.00,"221,792,054.23","14,892.68"
DKK,0.00,0.00,0.00,0.00
EUR,"4,588,587,004,603.74",0.00,"10,829,391,957,201.19","3,290,804.15"
GBP,"8,543,629,106.73",0.00,"15,630,068,292.27","125,020.27"
HKD,305.39,0.00,305.39,17.48


## RiskType cross bucket correlation

Now handle cross bucket correlation.
 - FX is .6 
 - IR is .5.  
 - Credit correlation is specifed in the regs
 - EQ is .15 for buckets 1-10, 0 otherwise
 - CM is .2 for buckets 1-10, 0 otherwise

In [68]:
MCVA = 1.25

def cross_correlation_values(df, correlation, zero):
    index = df.index.values
    corr = []
    for i in index:
        if i == zero:
            match = np.zeros_like(index, np.float64)
        else:
            match = np.ones_like(index, np.float64)*correlation
        corr.append(match)
    diag = np.array(corr)
    np.fill_diagonal(diag,1.0)
    return diag

def cross_correlation(df, correlation):
    diag = correlation*np.ones((df.shape[0],df.shape[0]))
    np.fill_diagonal(diag,1.0)
    return diag

Delta_Risk = pd.DataFrame( {
    'FX':MCVA*np.sqrt(bucket_delta_fx['K_b'].T.dot( cross_correlation(bucket_delta_fx, .6) ).dot(bucket_delta_fx['K_b'])),
    'IR':MCVA*np.sqrt(bucket_delta_IR['K_b'].T.dot( cross_correlation(bucket_delta_IR, .5) ).dot(bucket_delta_IR['K_b'])),
    'EQ':MCVA*np.sqrt(bucket_delta_eq['K_b'].T.dot( 
        cross_correlation_values(bucket_delta_eq, .15, 'Other sector') ).dot(bucket_delta_eq['K_b'])),
    'CM':MCVA*np.sqrt(bucket_delta_cm['K_b'].T.dot( 
        cross_correlation_values(bucket_delta_cm, .2, 'Other commodity') ).dot(bucket_delta_cm['K_b'])),
    'CR':MCVA*np.sqrt(bucket_delta_cr['K_b'].T.dot( CR_correlation ).dot(bucket_delta_cr['K_b']))
                           }, index=['Delta'] )

Vega_Risk =  pd.DataFrame( {
    'FX':MCVA*np.sqrt(bucket_vega_fx['K_b'].T.dot( cross_correlation(bucket_vega_fx, .6) ).dot(bucket_vega_fx['K_b'])),
    'IR':MCVA*np.sqrt(bucket_vega_ir['K_b'].T.dot( cross_correlation(bucket_vega_ir, .5) ).dot(bucket_vega_ir['K_b'])),
    'EQ':MCVA*np.sqrt(bucket_vega_eq['K_b'].T.dot( 
        cross_correlation_values(bucket_vega_eq, .15, 'Other sector') ).dot(bucket_vega_eq['K_b'])),
    'CM':MCVA*np.sqrt(bucket_vega_cm['K_b'].T.dot( 
        cross_correlation_values(bucket_vega_cm, .2, 'Other commodity') ).dot(bucket_vega_cm['K_b'])
                     ) if bucket_vega_cm['K_b'].any() else 0.0
    }, index=['Vega'] )

In [69]:
Final = pd.concat([Delta_Risk, Vega_Risk], sort=False).T.fillna(0.0)
Final

,Delta,Vega
FX,"284,082,877.55","8,413,249.40"
IR,"41,461,546.27","44,682,580.80"
EQ,"17,386.02","99,455.47"
CM,183.78,0.00
CR,"39,903,666.42",0.00


In [1]:
Final.sum(axis=1)

NameError: name 'Final' is not defined